<a href="https://colab.research.google.com/github/reic/colab_python/blob/main/%E9%8C%84%E9%9F%B3%E6%AA%94%E8%BD%89%E6%96%87%E5%AD%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 語音轉文字小工具

此工具採用 Python 開發，可應用於<font color="red">訪談錄音檔</font>轉文字、<font color="red">影片的字幕</font>字幕生成，及其它相關應用。

因為透過Google Colab 平台、Google的語音轉文字工具，完成語音轉文字的工作。只需要有 Google 帳號，即可具備執行此程式的環境，輔以簡單的設定，不會程式的使用者也可以完成相關的工作。

by 瑞課



## 1.安裝需求套件
* 文字轉語音套件
* 繁簡轉換套件

In [ ]:
# 執行安裝套件
!pip3 install SpeechRecognition
!pip install iNLP

## 2.掛載 google 雲端硬碟

請點選左側的<font color="#0000dd"> **檔案** </font>圖示，掛載 Google Drive 雲端硬碟

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 3.設定環境變數與函數預載

1. 取得 mp3 檔路徑
2. 設定 wav 切割檔的輸出路徑
3. 設定 txt 的輸出路徑
4. 若 wav, txt 的目錄存在，是否移除重建。預設：是y

In [ ]:
#@title 基礎環境設定
import os
import shutil
import speech_recognition as sr
import wave
import json
import numpy as np
from inlp.convert import chinese
 
#@markdown 錄音檔的位置
mp3Name= '/content/drive/MyDrive/\u524D\u77BB\u5C0F\u7D44/52_\u6C7A\u7B56\u652F\u63F4/\u7FA4\u7D44\u5DE5\u4F5C\u6703\u8B70/20210125/\u6703\u8B70\u8A18\u9304/210125_001.MP3' #@param {type:"string"}

#@markdown 設定錄音檔的分割大小，單位：秒。時間太長，轉文字的效果會較差。
CutTimeDef = 20 #@param {type:"integer"} 
#@markdown 設定 wav 切割檔的暫存目錄
wav_path='wav' #@param {type:"string"}
#@markdown 設定文字檔暫存目錄。將特定秒數(CutTimeDef)的音檔轉為文字
txt_path='txt' #@param {type:"string"}
 
workpath=os.path.dirname(mp3Name)
mp3Name=os.path.basename(mp3Name)
FileName = mp3Name[:-4]+".wav"
os.chdir(workpath)
#@markdown 若 wav_path, txt_path 目錄存在是否移除重建
chk='y' #@param ["y","n"]
 
def reset_dir(path):
    try:
        os.mkdir(path)
    except Exception:
        shutil.rmtree(path)
        os.mkdir(path)
 
def CutFile(FileName, target_path):
 
    # print("CutFile File Name is ", FileName)
    f = wave.open(FileName, "rb")
    params = f.getparams()    
    nchannels, sampwidth, framerate, nframes = params[:4]
    CutFrameNum = framerate * CutTimeDef
    # 讀取格式資訊
    # 一次性返回所有的WAV檔案的格式資訊，它返回的是一個組元(tuple)：聲道數, 量化位數（byte    單位）, 採
    # 樣頻率, 取樣點數, 壓縮型別, 壓縮型別的描述。wave模組只支援非壓縮的資料，因此可以忽略最後兩個資訊
 
    # print("CutFrameNum=%d" % (CutFrameNum))
    # print("nchannels=%d" % (nchannels))
    # print("sampwidth=%d" % (sampwidth))
    # print("framerate=%d" % (framerate))
    # print("nframes=%d" % (nframes))
 
    str_data = f.readframes(nframes)
    f.close()  # 將波形資料轉換成陣列
    # Cutnum =nframes/framerate/CutTimeDef
    # 需要根據聲道數和量化單位，將讀取的二進位制資料轉換為一個可以計算的陣列
    wave_data = np.frombuffer(str_data, dtype=np.short)
    wave_data.shape = -1, 2
    wave_data = wave_data.T
    temp_data = wave_data.T
    # StepNum = int(nframes/200)
    StepNum = CutFrameNum
    StepTotalNum = 0
    haha = 0
    while StepTotalNum < nframes:
        # for j in range(int(Cutnum)):
        # print("Stemp=%d" % (haha))
        SaveFile = "%s-%03d.wav" % (FileName[:-4], (haha+1))
        # print(FileName)
        if haha % 3==0:
          print("*",end='')
        temp_dataTemp = temp_data[StepNum * (haha):StepNum * (haha + 1)]
        haha = haha + 1
        StepTotalNum = haha * StepNum
        temp_dataTemp.shape = 1, -1
        temp_dataTemp = temp_dataTemp.astype(np.short)  # 開啟WAV文件
        f = wave.open(target_path+"/" + SaveFile, "wb")
        # 配置聲道數、量化位數和取樣頻率
        f.setnchannels(nchannels)
        f.setsampwidth(sampwidth)
        f.setframerate(framerate)
        # 將wav_data轉換為二進位制資料寫入檔案
        f.writeframes(temp_dataTemp.tobytes())
        f.close()
 
 
def VoiceToText(path, files, target_path):
  files.sort()
  for file in files:
    txt_file = "%s/%s.txt" % (target_path, file[:-4])
    if os.path.isfile(txt_file):
      continue
    with open("%s/%s.txt" % (target_path, file[:-4]), "w", encoding="utf-8") as f:
      f.write("%s:\n" % file)
      r = sr.Recognizer()  # 預設辨識英文
      with sr.WavFile(path+"/"+file) as source:  # 讀取wav檔
        audio = r.record(source)
      try:
        text = r.recognize_google(audio,language = "zh-tw")
        text = chinese.s2t(text)
        # r.recognize_google(audio)
        print(file)
        if len(text) == 0:
          print("===無資料==")
          continue
 
        print(text)
        f.write("%s \n\n" % text)
        if file == files[-1]:
            print("結束翻譯")
      except sr.RequestError as e:
        print("無法翻譯{0}".format(e))
        # 兩個 except 是當語音辨識不出來的時候 防呆用的
        # 使用Google的服務
      except LookupError:
        print("Could not understand audio")
      except sr.UnknownValueError:
        print("Error: 無法識別 Audio")
 
 
def texts_to_one(path, target_file):
    files = os.listdir(path)
    files.sort()
    files = [path+"/" + f for f in files if f.endswith(".txt")]
    with open(target_file, "w", encoding="utf-8") as f:
        for file in files:
            with open(file, "r", encoding='utf-8') as f2:
                f.write(f2.read())
    print("完成合併, 檔案位於 %s " % target_file)
 
 
def texts2otr(path, target_file, audio_name, timeperiod):
    template = '''<p><span class="timestamp" data-timestamp="{}.000000">{}</span>{}</p><p><br/></p>
    '''
    files = os.listdir(path)
    files.sort()
    content = ''
    files = [path+"/" + f for f in files if f.endswith(".txt")]
    with open(target_file, "w", encoding="utf-8") as f:
 
        for file in files:
            with open(file, "r", encoding="utf-8") as f2:
                txt = f2.read().split("\n")
                if len(txt) < 2:
                    continue
                times = int(txt[0].split("-")[1][:-5])*30
                secs, mins = times % 60, (times//60) % 60
                hours = (times//60)//60
                timeF = "{:02d}:{:02d}:{:02d}".format(hours, mins, secs)
                content += template.format(times, timeF, txt[1])
 
        output = {"text": content, "media": audio_name,
                  "media-time": timeperiod}
        f.write(json.dumps(output, ensure_ascii=False))

## 4.音頻轉換與切割

1. 將 mp3 轉成 wav 檔
2. 將音頻切割，並置於 wav_path 目錄下
3. 建立 txt_path ，做為語音判識的輸出檔




In [ ]:
#@title 執行音頻轉換與分割
 
print(" mp3 轉 wav 檔 ".center(100,'=')) 
os.system('{} -i {} {}'.format("ffmpeg",mp3Name, FileName))
print(" Wav 檔名為 {} ".format(FileName).center(96))
if chk == 'y':
    reset_dir(wav_path)
    reset_dir(txt_path)
# # Cut Wave Setting

print(" 音頻以每{}秒分割 ".format(CutTimeDef).center(94,'='))
CutFile(FileName, wav_path)
print("")
print(" 完成分割 ".center(100,'-'))

## 5.文字轉語音

In [ ]:
#@title 執行語音轉文字 (需要耗費不少時間)
#@\
files = os.listdir(wav_path)
VoiceToText(wav_path, files, txt_path)
 
target_txtfile = "{}.txt".format(FileName[:-4])
texts_to_one(txt_path, target_txtfile)
otr_file = "{}.otr".format(FileName[:-4])
with wave.open(FileName, "rb") as f:
    params = f.getparams()
texts2otr(txt_path, otr_file, FileName, params.nframes)

In [ ]:
#@title 列出合併的文字檔之檔名
#@markdown 將會形成 txt 和 [oTranscribe](https://otranscribe.com/) 網站使用的 otr 格式。輸出檔將置於上傳錄音檔同目錄。

#@markdown 若已知道檔名，不需要執行此區塊。
print(" 輸出檔名 ".center(100,'='))
print(target_txtfile)
print(otr_txtfile)

## 6.暫存檔、暫目錄清理

In [ ]:
#@title 暫存目錄、暫存檔清理，並選擇是否中斷Google 雲端硬碟
#@markdown <font color="#0000dd">wav_path, txt_path</font> 目錄存在是否移除重建
remove_tmp='y' #@param ["y","n"]

#@markdown <font color="#0000dd">Google 雲端硬碟</font> 連結是否中斷(預設:n)
umount_google='n' #@param ["y","n"]
if remove_tmp=='y':
  shutil.rmtree(wav_path)
  shutil.rmtree(txt_path)
  os.remove(FileName)
if umount_google=='y':
  drive.flush_and_unmount()